## Populate MOF with given molecule and initialize LAMMPS
- Read MOF xyz file
- Get size of the MOF
- Center MOF to size/2
- Add molecule
- Run packmol
- Convert xyz to cif

In [7]:
import os, shutil
from angstrom import Molecule
from packmol import Packmol

In [3]:
cells = {'BEDYEQ': [17.1430, 41.9680, 41.9680, 120.0, 90.0, 90.0],
         'IRMOF-1': [25.8320, 25.8320, 25.8320, 90.0, 90.0, 90.0],
         'UIO-66': [20.7004, 20.7004, 20.7004, 90.0, 90.0, 90.0]}

n_molecules = 1
molecule_padding = 5
packmol_dir = '/home/kutay/Documents/Research/DMMP/packmol'

In [4]:
for mof_name in cells:
    mof_file = os.path.join(packmol_dir, '%s-111.xyz' % mof_name)
    mof = Molecule(read=mof_file)
    
    # Replicate and get box size
    mof.set_cell(cells[mof_name])
    mof.cell.calculate_vertices()
    mof222 = mof.replicate([2, 2, 2])
    mof222.write(os.path.join(packmol_dir, '%s-222.xyz' % mof_name))
    box_size = [i * 2 for i in cells[mof_name][:3]]
    
    pmol = Packmol()
    pmol.options['tolerance'] = 3.5
    # Add mof
    mof_position = {'fixed': '%.1f %.1f %.1f 0. 0. 0.' % (box_size[0] / 2, box_size[1] / 2, box_size[2] / 2), 'centerofmass': ''}
    pmol.add_structure({'structure': '%s-222.xyz' % mof_name, 'number': '1', 'position': mof_position})
    
    # Add molecule
    mol_file = os.path.join(packmol_dir, 'dmma.xyz')
    mol_position = [molecule_padding] * 3 + [max(box_size) - molecule_padding] * 3
    mol_position = [5, 5, 5, 40, 40, 40]
    mol_position = {'inside box': '%s' % ' '.join([str(round(i, 1)) for i in mol_position])}
    pmol.add_structure({'structure': 'dmmp.xyz', 'number': n_molecules, 'position': mol_position})
    
    run_dir = os.path.join(packmol_dir, '%s' % mof_name)
    pmol.run(run_dir, packmol_dir)
    
    # Create cif file
    packed_xyz = Molecule(read=os.path.join(run_dir, pmol.options['output']))
    cell_center = mof.cell.vertices[0] + mof.cell.vertices[-1]
    packed_xyz.center(cell_center)
    packed_xyz.write(os.path.join(packmol_dir, '%s-dmmp.cif' % mof_name), cell=mof222.cell.to_list())
    
    pmol.clear()

### LAMMPS

In [8]:
from thermof import Simulation
from thermof import Parameters
lammps_dir = '/home/kutay/Documents/Research/DMMP/lammps'

In [ ]:
for mof_name in cells:
    sim_dir = os.path.join(lammps_dir, mof_name)
    cif_file = os.path.join(packmol_dir, '%s-dmmp.cif' % mof_name)
    
    simpar = Parameters()
    sim = Simulation(mof=os.path.abspath(cif_file), parameters=simpar)
    sim.set_dir(sim_dir)
    simpar.thermof['fix'] = ['MIN', 'NVT', 'NVE']
    simpar.thermof['min']['edif'] = 1e-3
    simpar.thermof['nvt']['steps'] = 500000
    simpar.thermof['nve']['steps'] = 1000000
    simpar.thermof['thermo_style'] = ['step', 'temp', 'press', 'pe', 'etotal', 'emol', 'epair', 'vol', 'lx', 'ly', 'lz']
    simpar.job['nodes'] = 1
    simpar.job['ppn'] = 28
    simpar.job['walltime'] = '36:00:00'
    simpar.job['name'] = '%s-dmmp' % mof_name
    sim.initialize()

Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1
Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1
I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
Molecules found in the framework, separating.
Files created! -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1
II. Updating Lammps input file -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/in.IRMOF-1-dmmp
Adding fixes: MIN | NVT | NVE
Updating simulation parameters...
III. Writing slurm job submission file -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/job.IRMOF-1-dmmp
Simulation parameters saved -> /home/kutay/Documents/Research/DMMP/lammps/IRMOF-1/simpar.yaml
Done!


/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 20 and 586 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 21 and 589 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2 and 604 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 5 and 605 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 561 and 614 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning:

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 6 and 2900 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 7 and 2901 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 8 and 2902 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 9 and 2903 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 10 and 2904 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarnin

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2364 and 4064 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2365 and 4066 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 1174 and 4068 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 1175 and 4069 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 1176 and 4070 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2948 and 4648 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3534 and 4649 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2949 and 4650 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3535 and 4651 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 1758 and 4652 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

...

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3829 and 4873 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3830 and 4874 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3831 and 4878 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3832 and 4879 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 3833 and 4880 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4437 and 4989 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4438 and 4990 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4439 and 4993 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4440 and 4994 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4441 and 4997 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4335 and 5091 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4336 and 5092 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4339 and 5093 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4340 and 5094 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 4343 and 5095 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2160 and 5204 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2161 and 5205 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2162 and 5206 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2163 and 5207 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:425: UserWarning: scaled_positions 2164 and 5208 are equivalent
  'are equivalent' % (kinds[ind], kind))
/home/kutay/anaconda3/lib/python3.5/site-packages/ase/spacegroup/spacegroup.py:4

Removing existing simulation directory -> /home/kutay/Documents/Research/DMMP/lammps/UIO-66
I. Writing Lammps input and data files...
No bonds reported in cif file - computing bonding..
